# Set up

In [13]:
# import libraries

import pandas as pd
import numpy as np

In [14]:
# create dataframe

data = {
    'time' : ['15:48:49', '15:46:10'],
    'type' : ['Sell/Close', 'Buy'],
    'instrument' : ['EUR/USD', 'EUR/USD'],
    'units' : [100000, 100000],
    'price' : [1.17791, 1.17775],
    'half_spread_cost' : [-9.5, -8.0],
    'spread_pips' : [1.90, 1.60]
}

trades = pd.DataFrame(data)

In [15]:
trades

,time,type,instrument,units,price,half_spread_cost,spread_pips
0,15:48:49,Sell/Close,EUR/USD,100000,1.17791,-9.5,1.9
1,15:46:10,Buy,EUR/USD,100000,1.17775,-8.0,1.6


# Calculate missing variables

In [16]:
# trade value = units * price
trades['trade_value'] = trades.units * trades.price

# trade profit = final value - initial value
trades['trade_profit'] = trades.trade_value[0] - trades.trade_value[1]

# trade profit in pips = (final_price - initial_price) * 10000
trades['trade_profit_pips'] = ( trades.price[0] - trades.price[1] ) * 10000

# pip value = units * 0.0001
trades['pip_value'] = trades.units[1] * 0.0001

# half spread costs = 0.5 * spread (pips) * pip value 
trades['half_spread_cost'] = 0.5 * trades.spread_pips * trades.pip_value * -1

# mid price (hypothetical) FV = Price - 0.5 * Spread (Pips) * 0.0001
# mid price (hypothetical) IV = Price - 0.5 * Spread (Pips) * 0.0001
trades['mid_price_hypo'] = np.nan

trades

,time,type,instrument,units,price,half_spread_cost,spread_pips,trade_value,trade_profit,trade_profit_pips,pip_value,mid_price_hypo
0,15:48:49,Sell/Close,EUR/USD,100000,1.17791,-9.5,1.9,117791.0,16.0,1.6,10.0,NaN
1,15:46:10,Buy,EUR/USD,100000,1.17775,-8.0,1.6,117775.0,16.0,1.6,10.0,NaN


In [17]:
trades.at[0, 'mid_price_hypo'] = trades.price[0] + 0.5 * trades.spread_pips[0] * 0.0001
trades.at[1, 'mid_price_hypo'] = trades.price[1] - 0.5 * trades.spread_pips[1] * 0.0001
trades

,time,type,instrument,units,price,half_spread_cost,spread_pips,trade_value,trade_profit,trade_profit_pips,pip_value,mid_price_hypo
0,15:48:49,Sell/Close,EUR/USD,100000,1.17791,-9.5,1.9,117791.0,16.0,1.6,10.0,1.178005
1,15:46:10,Buy,EUR/USD,100000,1.17775,-8.0,1.6,117775.0,16.0,1.6,10.0,1.177670


In [18]:
trades[['time', 'type', 'instrument', 'units', 'price', 'spread_pips', 'mid_price_hypo']]

,time,type,instrument,units,price,spread_pips,mid_price_hypo
0,15:48:49,Sell/Close,EUR/USD,100000,1.17791,1.9,1.178005
1,15:46:10,Buy,EUR/USD,100000,1.17775,1.6,1.177670


In [19]:
data_perf_attribution = {
    'profit_usd': {
        'gross_profit': np.nan,
        'trading_cost': np.nan,
        'net_profit': np.nan
    },
    'profit_pips': {
        'gross_profit': np.nan,
        'trading_cost': np.nan,
        'net_profit': np.nan
    },
    'return': {
        'gross_profit': np.nan,
        'trading_cost': np.nan,
        'net_profit': np.nan
    }
}

perf_att = pd.DataFrame(data_perf_attribution)

perf_att

,profit_usd,profit_pips,return
gross_profit,NaN,NaN,NaN
trading_cost,NaN,NaN,NaN
net_profit,NaN,NaN,NaN


> Label-based indexing: \
> .loc['column_name', 'row_name'] = can use ranges / slice \
> .at['column_name', 'row_name'] = faster for returning single cell

# Performance attribution

## Gross Profit

In [20]:
# gross profit (usd) = diff in mid-prices (FV-IV) * number of units
perf_att.at['gross_profit', 'profit_usd'] = (trades.mid_price_hypo[0] - trades.mid_price_hypo[1]) * trades.units[0]

# gross profit (pips) = diff in mid-prices (FV-IV) * 10000
perf_att.at['gross_profit', 'profit_pips'] = (trades.mid_price_hypo[0] - trades.mid_price_hypo[1]) * 10000

# return (ROI) = (profit / investment) * 100
perf_att.at['gross_profit', 'return'] = perf_att.at['gross_profit', 'profit_usd'] / trades.trade_value[1] * 100

perf_att

,profit_usd,profit_pips,return
gross_profit,33.5,3.35,0.028444
trading_cost,NaN,NaN,NaN
net_profit,NaN,NaN,NaN


## Trading costs

In [21]:
# total cost = sum(Half Spread Costs)
perf_att.at['trading_cost', 'profit_usd'] = abs(sum(trades.half_spread_cost))

# cost in pips = avg(profit in pips)
perf_att.at['trading_cost', 'profit_pips'] = np.mean(trades.spread_pips)

# impact of cost on initial investment = profit (in pips) / initial investment * 100
perf_att.at['trading_cost', 'return'] = perf_att.at['trading_cost', 'profit_pips'] / trades.trade_value[1] * 100

perf_att

,profit_usd,profit_pips,return
gross_profit,33.5,3.35,0.028444
trading_cost,17.5,1.75,0.001486
net_profit,NaN,NaN,NaN


## Net Profit

In [22]:
# net profit = gross profit - cost
perf_att.at['net_profit', 'profit_usd'] = perf_att.at['gross_profit', 'profit_usd'] - perf_att.at['trading_cost', 'profit_usd']

# profit (pips) = gross profit (pips) - cost (pips)
perf_att.at['net_profit', 'profit_pips'] = perf_att.at['gross_profit', 'profit_pips'] - perf_att.at['trading_cost', 'profit_pips']

# net profit (return) = gross profit (return) - cost (return)
perf_att.at['net_profit', 'return'] = perf_att.at['gross_profit', 'return'] - perf_att.at['trading_cost', 'return']

perf_att

,profit_usd,profit_pips,return
gross_profit,33.5,3.35,0.028444
trading_cost,17.5,1.75,0.001486
net_profit,16.0,1.60,0.026958


#  Notes

**Trading costs**: almost 75% of day traders lose money due to cost of exessive transactions \
\
**Mid price**: 'hypothetical' because can't buy or sell at mid price